In [1]:
# Importing the SQLite package
import sqlite3
# Importing error checking 
from sqlite3 import Error

import pandas as pd

In [5]:
# Establish database connection
connection = sqlite3.connect("northwind.db")

In [7]:
def execute_read_query(connection,query):
    cursor = connection.cursor()
    result = None 
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return  result
    except Error as e:
        print(f"The Error '{e}' occurred.")

In [17]:
cursor.execute('''with cte as (SELECT ContactName FROM Customers
        WHERE Address < 20)
select * from cte
''')
result = cursor.fetchall()
result

[('Thomas Hardy',),
 ('Laurence Lebihan',),
 ('Martine Rancé',),
 ('Annette Roulet',),
 ('Yoshi Tannamuri',),
 ('John Steel',),
 ('Jose Pavarotti',),
 ('Mary Saveley',)]

In [19]:
average_order_value = '''
    SELECT c.CustomerID,
       c.CompanyName,
       ROUND(AVG(ot.OrderTotal), 2) AS AverageOrderValue
  FROM Customers AS c
       JOIN
       (
           SELECT o.CustomerID,
                  SUM(od.UnitPrice * od.Quantity * (1 - od.Discount) ) AS OrderTotal
             FROM [Order Details] AS od
                  JOIN
                  Orders AS o ON od.OrderID = o.OrderID
            GROUP BY o.OrderID
       )
       AS ot ON c.CustomerID = ot.CustomerID
 GROUP BY c.CustomerID,
          c.CompanyName
 ORDER BY AverageOrderValue DESC;
 '''
first_query = execute_read_query(connection, average_order_value)

In [23]:
first_query[0:3]

[('FISSA', 'FISSA Fabrica Inter. Salchichas S.A.', 30448.65),
 ('PICCO', 'Piccolo und mehr', 30318.79),
 ('Val2 ', 'IT', 30316.21)]

In [33]:
first_query_df = pd.DataFrame(
    data = first_query,
    columns = ['CustomerID', 'CompanyName', 'AverageOrderValue']
)
first_query_df

,CustomerID,CompanyName,AverageOrderValue
0,FISSA,FISSA Fabrica Inter. Salchichas S.A.,30448.65
1,PICCO,Piccolo und mehr,30318.79
2,Val2,IT,30316.21
3,MORGK,Morgenstern Gesundkost,30051.07
4,PERIC,Pericles Comidas clásicas,30010.78
...,...,...,...
88,LEHMS,Lehmanns Marktstand,25205.69
89,QUEDE,Que Delícia,25196.45
90,ALFKI,Alfreds Futterkiste,24328.01
91,ERNSH,Ernst Handel,24288.01


In [43]:
with pd.option_context('display.max_rows', None):
    print(first_query_df)

   CustomerID                           CompanyName  AverageOrderValue
0       FISSA  FISSA Fabrica Inter. Salchichas S.A.           30448.65
1       PICCO                      Piccolo und mehr           30318.79
2       Val2                                     IT           30316.21
3       MORGK                Morgenstern Gesundkost           30051.07
4       PERIC             Pericles Comidas clásicas           30010.78
5       PARIS                     Paris spécialités           29850.85
6       THECR                       The Cracker Box           29748.41
7       MAISD                          Maison Dewey           29727.34
8       BOLID             Bólido Comidas preparadas           29659.69
9       SIMOB                         Simons bistro           29611.76
10      LAZYK                  Lazy K Kountry Store           29593.22
11      LACOR                  La corne d'abondance           29550.21
12      SANTG                         Santé Gourmet           29504.72
13    

In [37]:
customer_ranking = '''
    WITH CustomerAverages AS (
  SELECT
    o.CustomerID,
    SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS OrderTotal
  FROM Orders o
  JOIN [Order Details] od ON o.OrderID = od.OrderID
  GROUP BY o.OrderID
),
Ranked AS (
  SELECT
    c.CustomerID,
    c.CompanyName,
    ROUND(AVG(ca.OrderTotal), 2) AS AvgOrderValue,
    RANK() OVER (ORDER BY AVG(ca.OrderTotal) DESC) AS Rank
  FROM Customers c
  JOIN CustomerAverages ca ON c.CustomerID = ca.CustomerID
  GROUP BY c.CustomerID, c.CompanyName
)
SELECT *
FROM Ranked
WHERE Rank IN (2, 3, 5, 8, 12, 15, 17)
ORDER BY Rank;
'''
second_query = execute_read_query(connection, customer_ranking)
second_query[0:3]

[('PICCO', 'Piccolo und mehr', 30318.79, 2),
 ('Val2 ', 'IT', 30316.21, 3),
 ('PERIC', 'Pericles Comidas clásicas', 30010.78, 5)]

In [39]:
second_query_df = pd.DataFrame(
    data = second_query,
    columns = ['CustomerID', 'CompanyName', 'AvgOrderValue', 'Rank']
)
second_query_df

,CustomerID,CompanyName,AvgOrderValue,Rank
0,PICCO,Piccolo und mehr,30318.79,2
1,Val2,IT,30316.21,3
2,PERIC,Pericles Comidas clásicas,30010.78,5
3,MAISD,Maison Dewey,29727.34,8
4,LACOR,La corne d'abondance,29550.21,12
5,BSBEV,B's Beverages,29305.31,15
6,VAFFE,Vaffeljernet,28894.04,17


In [48]:
# Query: compute total for each order
order_totals_query = """
SELECT
    o.OrderID,
    o.CustomerID,
    o.OrderDate,
    SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS OrderTotal
FROM Orders o
JOIN [Order Details] od ON o.OrderID = od.OrderID
GROUP BY o.OrderID, o.CustomerID, o.OrderDate
"""
order_totals = execute_read_query(connection, order_totals_query)

In [50]:
order_totals_df = pd.DataFrame(
    data = order_totals,
    columns = ['OrderID', 'CustomerID', 'OrderDate', 'OrderTotal']
)
order_totals_df.head(3)

,OrderID,CustomerID,OrderDate,OrderTotal
0,10248,VINET,2016-07-04,440.0
1,10249,TOMSP,2016-07-05,1863.4
2,10250,HANAR,2016-07-08,1552.6


In [52]:
# Query: aggregate customer orders with counts, totals, averages, and ranks
customer_agg_query = """
WITH OrderTotals AS (
    SELECT
        o.OrderID,
        o.CustomerID,
        SUM(od.UnitPrice * od.Quantity * (1 - od.Discount)) AS OrderTotal
    FROM Orders o
    JOIN [Order Details] od ON o.OrderID = od.OrderID
    GROUP BY o.OrderID, o.CustomerID
)
SELECT
    c.CustomerID,
    c.CompanyName,
    COUNT(ot.OrderID) AS OrderCount,
    SUM(ot.OrderTotal) AS TotalRevenue,
    AVG(ot.OrderTotal) AS AvgOrderValue,
    RANK() OVER (ORDER BY AVG(ot.OrderTotal) DESC) AS AOV_Rank
FROM Customers c
JOIN OrderTotals ot ON ot.CustomerID = c.CustomerID
GROUP BY c.CustomerID, c.CompanyName
"""
customer_agg = execute_read_query(connection, customer_agg_query)

In [86]:
customer_agg_df = pd.DataFrame(
    data = customer_agg,
    columns = ['CustomerID', 'CompanyName', 'OrderCount', 'TotalRevenue', 'AvgOrderValue', 'AOV_Rank']
)
customer_agg_df.head(20)

,CustomerID,CompanyName,OrderCount,TotalRevenue,AvgOrderValue,AOV_Rank
0,FISSA,FISSA Fabrica Inter. Salchichas S.A.,155,4.719540e+06,30448.646452,1
1,PICCO,Piccolo und mehr,177,5.366426e+06,30318.792486,2
2,Val2,IT,159,4.820277e+06,30316.205535,3
3,MORGK,Morgenstern Gesundkost,178,5.349090e+06,30051.065112,4
4,PERIC,Pericles Comidas clásicas,169,5.071823e+06,30010.784201,5
5,PARIS,Paris spécialités,161,4.805988e+06,29850.854969,6
6,THECR,The Cracker Box,168,4.997732e+06,29748.407202,7
7,MAISD,Maison Dewey,162,4.815829e+06,29727.341512,8
8,BOLID,Bólido Comidas preparadas,182,5.398064e+06,29659.694725,9
9,SIMOB,Simons bistro,161,4.767494e+06,29611.763587,10


In [98]:
# Top 10 Customers by Average Order Value
customer_agg_df[['CustomerID', 'CompanyName', 'OrderCount','TotalRevenue', 'AvgOrderValue', 'AOV_Rank']].sort_values("AvgOrderValue", ascending = False).head(10)

,CustomerID,CompanyName,OrderCount,TotalRevenue,AvgOrderValue,AOV_Rank
0,FISSA,FISSA Fabrica Inter. Salchichas S.A.,155,4.719540e+06,30448.646452,1
1,PICCO,Piccolo und mehr,177,5.366426e+06,30318.792486,2
2,Val2,IT,159,4.820277e+06,30316.205535,3
3,MORGK,Morgenstern Gesundkost,178,5.349090e+06,30051.065112,4
4,PERIC,Pericles Comidas clásicas,169,5.071823e+06,30010.784201,5
5,PARIS,Paris spécialités,161,4.805988e+06,29850.854969,6
6,THECR,The Cracker Box,168,4.997732e+06,29748.407202,7
7,MAISD,Maison Dewey,162,4.815829e+06,29727.341512,8
8,BOLID,Bólido Comidas preparadas,182,5.398064e+06,29659.694725,9
9,SIMOB,Simons bistro,161,4.767494e+06,29611.763587,10


In [100]:
# Top 10 Customers by Total Revenue
customer_agg_df[['CustomerID', 'CompanyName', 'OrderCount', 'TotalRevenue', 'AvgOrderValue', 'AOV_Rank']].sort_values("TotalRevenue", ascending = False).head(10)

,CustomerID,CompanyName,OrderCount,TotalRevenue,AvgOrderValue,AOV_Rank
14,BSBEV,B's Beverages,210,6.154115e+06,29305.311143,15
18,HUNGC,Hungry Coyote Import Store,198,5.698024e+06,28777.897323,19
19,RANCH,Rancho grande,194,5.559110e+06,28655.206598,20
42,GOURL,Gourmet Lanchonetes,202,5.552310e+06,27486.682203,43
22,ANATR,Ana Trujillo Emparedados y helados,195,5.534357e+06,28381.316154,23
56,RICAR,Ricardo Adocicados,203,5.524517e+06,27214.370985,57
27,FOLIG,Folies gourmandes,195,5.505503e+06,28233.347949,28
20,LETSS,Let's Stop N Shop,191,5.462198e+06,28597.895406,21
68,LILAS,LILA-Supermercado,203,5.437438e+06,26785.410542,69
58,PRINI,Princesa Isabel Vinhos,200,5.436771e+06,27183.852750,59


In [104]:
# Top 10 Customers by Order Count
customer_agg_df[['CustomerID', 'CompanyName', 'OrderCount', 'TotalRevenue', 'AvgOrderValue', 'AOV_Rank']].sort_values("OrderCount", ascending = False).head(10)

,CustomerID,CompanyName,OrderCount,TotalRevenue,AvgOrderValue,AOV_Rank
14,BSBEV,B's Beverages,210,6154115.340,29305.311143,15
56,RICAR,Ricardo Adocicados,203,5524517.310,27214.370985,57
68,LILAS,LILA-Supermercado,203,5437438.340,26785.410542,69
42,GOURL,Gourmet Lanchonetes,202,5552309.805,27486.682203,43
58,PRINI,Princesa Isabel Vinhos,200,5436770.550,27183.852750,59
18,HUNGC,Hungry Coyote Import Store,198,5698023.670,28777.897323,19
67,TORTU,Tortuga Restaurante,197,5278083.970,26792.304416,68
22,ANATR,Ana Trujillo Emparedados y helados,195,5534356.650,28381.316154,23
27,FOLIG,Folies gourmandes,195,5505502.850,28233.347949,28
19,RANCH,Rancho grande,194,5559110.080,28655.206598,20


In [94]:
# Desired ranks
target_ranks = [2, 3, 5, 8, 12, 15, 17]

# Filter rows and create a new DataFrame
selected_customers = customer_agg_df[customer_agg_df["AOV_Rank"].isin(target_ranks)].copy()

# Sort rows by AOV_Rank
selected_customers = selected_customers.sort_values("AOV_Rank")
selected_customers

,CustomerID,CompanyName,OrderCount,TotalRevenue,AvgOrderValue,AOV_Rank
1,PICCO,Piccolo und mehr,177,5366426.270,30318.792486,2
2,Val2,IT,159,4820276.680,30316.205535,3
4,PERIC,Pericles Comidas clásicas,169,5071822.530,30010.784201,5
7,MAISD,Maison Dewey,162,4815829.325,29727.341512,8
11,LACOR,La corne d'abondance,155,4580281.870,29550.205613,12
14,BSBEV,B's Beverages,210,6154115.340,29305.311143,15
16,VAFFE,Vaffeljernet,171,4940880.785,28894.039678,17
